In [ ]:
!pip install langchain_groq tavily-python langchain-community langchain-openai langchainhub --quiet

In [ ]:
import os
from google.colab import userdata
os.environ["TAVILY_API_KEY"] =userdata.get('TAVILY_API_KEY')
os.environ["GROQ_API_KEY"] =userdata.get('GROQ_API_KEY')

In [ ]:
from groq import Groq

client = Groq()
completion = client.chat.completions.create(
    model="llama-3.1-70b-versatile",
    messages=[
        {
            "role": "user",
            "content": "Who is LLM"
        }
    ],
    temperature=1,
    max_tokens=1024,
    top_p=1,
    stream=True,
    stop=None,
)

for chunk in completion:
    print(chunk.choices[0].delta.content or "", end="")


"LLM" can refer to several things, but in the context of recent advancements in artificial intelligence, I'm assuming you're asking about Large Language Models.

A Large Language Model (LLM) is a type of artificial intelligence (AI) model that uses natural language processing (NLP) to generate human-like text responses. These models are trained on vast amounts of text data, which enables them to learn patterns, relationships, and context in language.

LLMs are designed to process and understand human language, generate text, and respond to questions or prompts. They're often used in chatbots, virtual assistants, language translation, text summarization, and other applications where human-like language understanding and generation are required.

Some well-known examples of LLMs include:

1. OpenAI's GPT models (e.g., GPT-3, GPT-4)
2. Google's BERT and its variants (e.g., RoBERTa, DeBERTa)
3. Meta AI's LLaMA
4. Microsoft's Turing models

These models have made significant strides in rece

In [ ]:
class Agent:
  def __init__(self,client,system):
      self.client = client
      self.system = system
      self.messages=[]
      if self.system is not None:
          self.messages.append({"role":"system","content":self.system})
  def __call__(self, messages="") :
      if messages:
          self.messages.append({"role":"user","content":messages})
      result = self.execute()
      self.messages.append({"role":"assistant","content":result})
      return result
  def execute(self):
      completion = client.chat.completions.create( model="llama-3.1-70b-versatile",
                    messages=self.messages )
      return completion.choices[0].message.content

In [ ]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 47 * 71 / 5
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

tavily_search:
e.g. tavily_search: Django
Search for that term in the internet and return the result

Always look things up on Wikipedia or tavily_search if you have the opportunity to do so.

Example session:

Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:

Observation: France is a country. The capital is Paris.

You then output:

Answer: The capital of France is Paris
""".strip()

In [ ]:
prompt1 = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

tavily_search:
e.g. tavily_search: Django
Search for that term in the internet and return the result

Always look things up on tavily_search if you have the opportunity to do so.

Example session:

Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:

Observation: France is a country. The capital is Paris.

You then output:

Answer: The capital of France is Paris
""".strip()

In [ ]:
# Tools
import re
import httpx

def calculate(input):
  return eval(input)

def wikipedia(q):
    return httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": q,
        "format": "json"
    }).json()["query"]["search"][0]["snippet"]

from tavily import TavilyClient
def tavily_search(q):
  client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])
  response = client.search(q)
  return response

In [ ]:
Alpha = Agent(client,prompt1)



Thought: The observation indicates that Prabhas is an Indian actor known mononymously by his first name. I would like to know more about his background and filmography to provide a more comprehensive answer.

Action: tavily_search: Prabhas filmography

PAUSE


[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\ntavily_search:\ne.g. tavily_search: Django\nSearch for that term in the internet and return the result\n\nAlways look things up on tavily_search if you have the opportunity to do so.\n\nExample session:\n\nQuestion: What is the capital of France?\nThought: I should look up France on Wikipedia\nAction: wikipedia: France\nPAUSE\n\nYou will be called again with this:\n\nObservation: France is a country. The capital is Paris.\n\nYou then output:\n\nAnswer: The capi

In [ ]:
Beta= Alpha()
print(Beta)

(I would have requested wikipedia if I could, however I can only use either calculate, or tavily_search, both of which should give a somewhat similar result to wikipedia so I will proceed with tavily_search)


In [ ]:
Alpha("Who is Prabhas")

'Thought: I should look up Prabhas on Wikipedia to gather information about him.\n\nAction: wikipedia: Prabhas\nPAUSE'

In [ ]:
Alpha.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nwikipedia:\ne.g. wikipedia: Django\nReturns a summary from searching Wikipedia\n\ntavily_search:\ne.g. tavily_search: Django\nSearch for that term in the internet and return the result\n\nAlways look things up on Wikipedia or tavily_search if you have the opportunity to do so.\n\nExample session:\n\nQuestion: What is the capital of France?\nThought: I should look up France on Wikipedia\nAction: wikipedia: France\nPAUSE\n\nYou will be called again with this:\n\n

Now we got he first step i.e the thought for the user query. The next step is we need to use this Thought as the input to get the Action.

In [ ]:
Beta= Alpha()
print(Beta)

In [ ]:
Cheta= wikipedia("Prabhas")
Cheta1=Alpha(f"Observation:{C}")
print(Cheta1)

Thought: Based on the observation, it seems I got some basic information about Prabhas but I'd like to know more. I'd like to find out the type of movies he has worked on and his achievements.

Action: wikipedia: Prabhas#Career
PAUSE


In [ ]:
Alpha.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nwikipedia:\ne.g. wikipedia: Django\nReturns a summary from searching Wikipedia\n\ntavily_search:\ne.g. tavily_search: Django\nSearch for that term in the internet and return the result\n\nAlways look things up on Wikipedia or tavily_search if you have the opportunity to do so.\n\nExample session:\n\nQuestion: What is the capital of France?\nThought: I should look up France on Wikipedia\nAction: wikipedia: France\nPAUSE\n\nYou will be called again with this:\n\n

In [ ]:
res=""" Thought: The summary provides information about Prabhas, including his full name and birthdate, but it does not provide a detailed overview of his career and life.

Action: wikipedia: Prabhas

PAUSE"""
action_re = re.findall(r'Action: ([a-z_]+): (.+)',res,re.IGNORECASE)

In [ ]:
action_re

[('wikipedia', 'Prabhas')]

In [ ]:
import re

def agent_loop(max_iterations,prompt,query):
  agent=Agent(client,prompt)
  tools=['calculate','tavily_search']#'wikipedia',

  next_prompt = query

  i=0
  while i < max_iterations:
    i+=1
    result = agent(next_prompt)
    print("Result",result)
    if "PAUSE" in result and "Action" in result:
      action_re = re.findall(r'Action: ([a-z_]+): (.+)',result,re.IGNORECASE)
      chosen_tool =action_re[0][0]
      tool_input=action_re[0][1]
      if chosen_tool in tools:
        print(" -- running {} {}".format(chosen_tool, tool_input))
        result_tool=eval(f"{chosen_tool}('{tool_input}')")
        print("Observation:", result_tool)
        next_prompt = "Observation: {}".format(result_tool)

      else:
        next_prompt= "Observation: Tool not found"
        raise Exception("Unknown action: {}: {}".format(chosen_tool, tool_input))
      print(next_prompt)
      continue
    if "Answer" in result:
      break



In [ ]:
agent_loop(max_iterations=10,prompt=prompt1,query="Who is Prabhas")

Result Thought: I've heard of Prabhas but I don't know much about him, I should look him up on the internet to find out more information.

Action: tavily_search: Prabhas
PAUSE
 -- running tavily_search Prabhas
Observation: {'query': 'Prabhas', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Prabhas - Simple English Wikipedia, the free encyclopedia', 'url': 'https://simple.wikipedia.org/wiki/Prabhas', 'content': 'Prabhas is an Indian actor who works mainly in Telugu cinema industry. He is one of the highest-paid actors in Indian cinema and starred in the Baahubali franchise, which became the first Indian film to gross ₹1000 crores.', 'score': 0.99971753, 'raw_content': None}, {'title': 'Prabhas - IMDb', 'url': 'https://www.imdb.com/name/nm1659141/', 'content': 'Prabhas. Actor: Baahubali: The Beginning. Prabhas is an Indian actor who works in Telugu cinema. One of the highest-paid and highest-grossing actor in Indian cinema. Prabhas is hailed as the "Fir

In [ ]:
agent_loop(
    max_iterations=10,
    prompt=prompt1,
    query="Who is Prabhas, Give her best works"
)

Result Thought: Prabhas seems like an Indian name, but I'm not familiar with the person. I should look up Prabhas on the internet to find out who he is and what he does. Also, the person is referred to as "her", but the name seems more commonly associated with a male, so I will look up both options.

Action: tavily_search: Prabhas
PAUSE
 -- running tavily_search Prabhas
Observation: {'query': 'Prabhas', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Prabhas (@actorprabhas) • Instagram photos and videos', 'url': 'https://www.instagram.com/actorprabhas/', 'content': '13M Followers, 23 Following, 232 Posts - See Instagram photos and videos from Prabhas (@actorprabhas)', 'score': 0.9996898, 'raw_content': None}, {'title': 'Prabhas filmography and awards - Wikipedia', 'url': 'https://en.wikipedia.org/wiki/Prabhas_filmography_and_awards', 'content': 'Prabhas filmography and awards. Prabhas is an Indian actor who predominantly works in Telugu cinema. He made